In [ ]:
%pip install zstandard pandas plotly scikit-learn

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=fd1eba0c287e5a6cc7306f1e890d203e432d40496ec2c92445083569d89cd4c1
  Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
import os
import requests
from pathlib import Path
from plotly.subplots import make_subplots
import plotly.graph_objs as go
from zstandard import ZstdCompressionWriter, ZstdDecompressor
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from numpy import array2string
from datetime import datetime

In [ ]:
# Local instance
import plotly.io as pio
# Set plotly render
pio.renderers.default = "colab"
%cd "/tf/work"

/tf/work


In [ ]:
# Download all model versions
print(f'Downloading fyp-forecasting-models-1_18.zip')
r = requests.get('https://files.nekoul.com/pub/fyp-forecasting-models-1_18.zip')
if not r.ok:
  print('Unable to download the archieve')
  exit(128)

with open('models-1_18.zip', 'wb') as f:
    f.write(r.content)

!unzip -nq models-1_18.zip
print(f'Extracted all models')

Extracted all models


In [ ]:
!ls -al model

In [ ]:
# Google Colab instance
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
# Create a working folder and cd to it.
!mkdir -p "/content/drive/MyDrive/Courses/EIE/Year 4/FYP"
%cd "/content/drive/MyDrive/Courses/EIE/Year 4/FYP"

Mounted at /content/drive
/content/drive/MyDrive/Courses/EIE/Year 4/FYP


In [ ]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
DATASET_FILE_PATH = 'hk_opendata_journey_data.csv'
if not Path(DATASET_FILE_PATH).exists():
  print(f'Downloading hk_opendata_journey_data.csv.zst')
  r = requests.get('https://files.nekoul.com/pub/hk_opendata_journey_data.csv.zst')
  if not r.ok:
    print('Unable to download the datasets')
    exit(128)

  with open(DATASET_FILE_PATH, 'wb+') as f:
    print(f'Decompressing {DATASET_FILE_PATH}')
    dctx = ZstdDecompressor()
    decompressor = dctx.stream_writer(f)
    decompressor.write(r.content)
    print(f'Decompression done')

In [ ]:
df = pd.read_csv(DATASET_FILE_PATH)
feature_keys = [
  'H1-CH', 'H1-EH', 'H11-CH', 'H11-EH', 'H2-CH', 'H2-EH', 'H2-WH', 'H3-CH', 'H3-WH', 'H4-CH', 'H4-EH', 'H4-WH',
  'H5-CH', 'H5-EH', 'H5-WH', 'K01-CH', 'K01-WH', 'K02-CH', 'K02-EH', 'K03-CH', 'K03-EH', 'K03-WH', 'K04-CH', 'K04-WH',
  'K05-CH', 'K05-EH', 'K06-CH', 'K06-WH'
]
df.index = pd.to_datetime(df['timestamp'])
df.drop('timestamp', axis=1, inplace=True)

In [ ]:
def plot(data, title: str = '', *,
         xaxis_title='Time', yaxis_title='Journey time', yaxis_range: list = None, rows=1, cols=1,
         names: list = None, subplot_pos: list = None, subplot_titles: list = None, indexes: list = None):
  fig = make_subplots(rows=rows, cols=cols, subplot_titles=subplot_titles)
  for idx, s in enumerate(data):
    pos = subplot_pos[idx] if subplot_pos is not None else idx
    index = indexes[idx] if indexes is not None else None
    name = names[idx] if names is not None else None
    row = pos // rows
    col = pos % cols + 1
    fig.add_trace(
      go.Scatter(x=index, y=s, name=name),
      row = pos // (rows + 1) + 1,
      col = pos % cols + 1,
    )
    fig.update_xaxes(title_text=xaxis_title, row=row, col=col)
    fig.update_yaxes(title_text=yaxis_title, row=row, col=col)

  fig.update_layout(
    title={
      'text': title,
      'x': 0.5,
    },
    yaxis_range=yaxis_range
  )

  fig.show()

In [ ]:
# Filter invalid time
skip_date = pd.to_datetime("2022-07-11")
df = df[df.index > skip_date]
df = df.resample('5Min').interpolate(method='time').iloc[1:]
df['week_day'] = df.index.dayofweek.values
df['hour'] = df.index.hour.values
df['minute'] = df.index.minute.values

In [ ]:
n_feature = 4

cht = df[['K02-CH', 'week_day', 'hour', 'minute']]
cht = cht[df['K02-CH'] != -1]

cht.shape

(35421, 4)

In [ ]:
# Smooth traffic data by move average
cht['K02-CH'] = cht['K02-CH'].rolling(6).mean().shift(periods=-2).fillna(4.8)

In [ ]:
cht

,K02-CH,week_day,hour,minute
timestamp,,,,
2022-03-11 00:05:00,4.800000,4,0,5
2022-03-11 00:10:00,4.800000,4,0,10
2022-03-11 00:15:00,4.800000,4,0,15
2022-03-11 00:20:00,4.000000,4,0,20
2022-03-11 00:25:00,4.000000,4,0,25
...,...,...,...,...
2022-11-10 23:35:00,5.333333,3,23,35
2022-11-10 23:40:00,5.166667,3,23,40
2022-11-10 23:45:00,5.000000,3,23,45


In [ ]:
# plot([
#   cht,
#   cht_5m,
#   cht_10m,
#   cht_30m,
# ], 'K02-CH', rows=2, cols=2, subplot_titles=['1 Min', '5 Min', '10 Min', '30 Min'])
plot([cht['K02-CH']], "K02-CH", indexes=[cht.index, cht.index], subplot_pos=[0, 0, 0], names=['K02-CH'])
# plot([cht['K02-CH']], indexs=[cht.index])

In [ ]:
# def splint_and_normalize_dataset(data, *,
#                                  train_ratio=0.7, val_ratio=0.2) -> (pd.Series, pd.Series, pd.Series):
#   data_n = data.shape[0]
#   train_n = int(data_n * train_ratio)
#   val_n = int(data_n * val_ratio)

#   train_data: pd.Series = data.iloc[:train_n]
#   # Normalize data
#   mean = train_data.mean()
#   std = train_data.std()
#   train_data = (train_data - mean) / std
#   val_data = (data.iloc[train_n:train_n + val_n] - mean) / std
#   test_data = (data.iloc[train_n + val_n:] - mean) / std
#   return train_data, val_data, test_data

In [ ]:
# train, val, test = splint_and_normalize_dataset(cht_5m)
# Split data into test/validate/test dataset (70/20/10)
data_n = cht.shape[0]
train_n = int(data_n * 0.70)
val_n = int(data_n * 0.20)

# mean = cht['K02-CH'].mean()
# std = cht['K02-CH'].std()
# train_data = cht.iloc[:train_n]
# train_data.loc[:,'K02-CH'] = (train_data['K02-CH'] - mean) / std
# val_data = cht.iloc[train_n:train_n + val_n]
# val_data.loc[:,'K02-CH'] = (val_data['K02-CH'] - mean) / std
# test_data = cht.iloc[train_n + val_n:]
# test_data.loc[:,'K02-CH'] = (test_data['K02-CH'] - mean) / std

# mm = MinMaxScaler(feature_range=(0, 1))
# data = mm.fit_transform(cht)
data = cht.to_numpy()
train_data = data[:train_n]
val_data = data[train_n:train_n + val_n]
test_data = data[train_n + val_n:]

print(f'Train data shape={train_data.shape}')
print(f'Validation data shape={val_data.shape}')
print(f'Test data shape={test_data.shape}')
print(f'index shape={cht.index.shape}')
plot([train_data[:,0], val_data[:,0], test_data[:,0]], 'K02-CH', rows=1, cols=1,
     names=['Train', 'Validation', 'Test'], subplot_pos=[0, 0, 0], indexes=[cht.index[:train_n], cht.index[train_n:train_n + val_n], cht.index[train_n + val_n:]])

Train data shape=(24794, 4)
Validation data shape=(7084, 4)
Test data shape=(3543, 4)
index shape=(35421,)


In [ ]:
def make_windows(data, n_steps = 12 * 12, n_horizon = 12 * 3, batch_size = 256, shift = 1, shuffle_size = 500):
  # Use the previous data points to predict the next n_horizon data points
  window = n_steps + n_horizon
  ds = tf.data.Dataset.from_tensor_slices(data)

  # Create the window combined the steps and horizon
  ds = ds.window(window, shift=shift, drop_remainder=True)
  # window() return nested dataset of windows but a regular dataset containing tensors is needed
  ds = ds.flat_map(lambda x : x.batch(window))
  if shuffle_size > 0:
    ds = ds.shuffle(shuffle_size)
  # Extract the features and labels from each windows
  ds = ds.map(lambda x : (x[:-n_horizon], x[-n_horizon:, :1]))
  # Batch the dataset
  ds = ds.batch(batch_size).prefetch(1)
  
  return ds

In [ ]:
# Prediction horizon
# Use the past n_steps  data pointsto predict the next n_horizon data points
n_steps = 12 * 6
n_horizon = 12 * 3

In [ ]:
# WINDOW_SIZE=24
# train_inputs, train_labels = make_dataset(train, WINDOW_SIZE)
# val_inputs, val_labels = make_dataset(val, WINDOW_SIZE)
# test_inputs, test_labels = make_dataset(test, WINDOW_SIZE)
# train_inputs.shape, train_labels.shape, val_inputs.shape, val_labels.shape, test_inputs.shape, test_labels.shape

# Window config
batch_size = 128
shift = 1

train = make_windows(train_data, n_steps, n_horizon, batch_size, shift)
val = make_windows(val_data, n_steps, n_horizon, batch_size, shift)
test = make_windows(test_data, n_steps, n_horizon, batch_size, shift)

for idx, (x,y) in enumerate(train):
    print("feature shape=", x.numpy().shape)
    print("label shape=", y.numpy().shape)
    break
print(f"train spec={train.element_spec}")
print(f"val spec={val.element_spec}")
print(f"test spec={test.element_spec}")

feature shape= (128, 72, 4)
label shape= (128, 36, 1)
train spec=(TensorSpec(shape=(None, None, 4), dtype=tf.float64, name=None), TensorSpec(shape=(None, None, 1), dtype=tf.float64, name=None))
val spec=(TensorSpec(shape=(None, None, 4), dtype=tf.float64, name=None), TensorSpec(shape=(None, None, 1), dtype=tf.float64, name=None))
test spec=(TensorSpec(shape=(None, None, 4), dtype=tf.float64, name=None), TensorSpec(shape=(None, None, 1), dtype=tf.float64, name=None))


In [ ]:
from keras.models import Sequential
from keras.layers import InputLayer, LSTM, Dense, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Reshape, Flatten, Dropout, Bidirectional
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.losses import MeanSquaredError, MeanAbsolutePercentageError, MeanAbsoluteError as MeanAbsoluteErrorLoss, Huber
from keras.metrics import RootMeanSquaredError, MeanAbsoluteError
from keras.optimizers import Adam
from keras.models import load_model

In [ ]:
model_title = "CNN-LSTM (12h-to-30m, Huber, Batch 128, 4CNN)"
model_name = "v18"

In [ ]:
# CNN-LSTM model
cnn_lstm_model = Sequential()
# Convolutional layer with 128 filters with the size of 3
cnn_lstm_model.add(Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(n_steps, n_feature)))
# Convolutional layer with 128 filters with the size of 3
cnn_lstm_model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
# Convolutional layer with 128 filters with the size of 3
cnn_lstm_model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
# MaxPooling2D layer with kernel size of 3
cnn_lstm_model.add(MaxPooling1D(pool_size=3))
# Dropout with the possibility of 0.8
cnn_lstm_model.add(Dropout(0.8))
# LSTM layer with 200 unit and use return_sequence (pass the output of each time step to the next layer)
cnn_lstm_model.add(Bidirectional(LSTM(400)))
# Dense Layer with 32 neutrons
cnn_lstm_model.add(Dense(32, 'relu'))
# Output layer to output the next n_horizon time steps
cnn_lstm_model.add(Dense(n_horizon, 'linear'))
cnn_lstm_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 70, 128)           1664      
                                                                 
 conv1d_4 (Conv1D)           (None, 68, 128)           49280     
                                                                 
 conv1d_5 (Conv1D)           (None, 66, 128)           49280     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 22, 128)          0         
 1D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 22, 128)           0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 800)              1692800   
 nal)                                                 

In [ ]:
epoch_n = 100
learning_rate = 0.001

checkpoint = ModelCheckpoint(f'model/{model_name}', save_best_only=True)
early_stop = EarlyStopping(monitor='val_loss', patience=15)
# Comment this line if train a new model
#CNN_LSTM_loaded_model = load_model('model/v6')
cnn_lstm_model.compile(loss=Huber(), optimizer=Adam(learning_rate=learning_rate), metrics=[MeanAbsoluteError()])

In [ ]:
history = cnn_lstm_model.fit(train, validation_data=val, epochs=epoch_n, callbacks=[checkpoint, early_stop])

Epoch 1/100
193/193 [==============================] - 5s 27ms/step - loss: 0.7034 - mean_absolute_error: 1.0603 - val_loss: 0.9974 - val_mean_absolute_error: 1.3788
Epoch 2/100
193/193 [==============================] - 5s 28ms/step - loss: 0.6924 - mean_absolute_error: 1.0488 - val_loss: 1.0176 - val_mean_absolute_error: 1.3993
Epoch 3/100
193/193 [==============================] - 5s 27ms/step - loss: 0.6721 - mean_absolute_error: 1.0267 - val_loss: 1.1826 - val_mean_absolute_error: 1.5707
Epoch 4/100
193/193 [==============================] - 5s 27ms/step - loss: 0.7295 - mean_absolute_error: 1.0874 - val_loss: 1.0587 - val_mean_absolute_error: 1.4465
Epoch 5/100
193/193 [==============================] - 5s 28ms/step - loss: 0.6756 - mean_absolute_error: 1.0301 - val_loss: 1.0154 - val_mean_absolute_error: 1.3950
Epoch 6/100
193/193 [==============================] - 5s 27ms/step - loss: 0.6826 - mean_absolute_error: 1.0377 - val_loss: 1.0313 - val_mean_absolute_error: 1.4148
Epoc

INFO:tensorflow:Assets written to: model/v18/assets


INFO:tensorflow:Assets written to: model/v18/assets


193/193 [==============================] - 16s 82ms/step - loss: 0.6808 - mean_absolute_error: 1.0355 - val_loss: 0.9689 - val_mean_absolute_error: 1.3440
Epoch 10/100
193/193 [==============================] - 5s 27ms/step - loss: 0.6652 - mean_absolute_error: 1.0189 - val_loss: 1.2916 - val_mean_absolute_error: 1.6764
Epoch 11/100
193/193 [==============================] - 5s 27ms/step - loss: 0.7604 - mean_absolute_error: 1.1214 - val_loss: 0.9844 - val_mean_absolute_error: 1.3570
Epoch 12/100
193/193 [==============================] - 5s 27ms/step - loss: 0.6591 - mean_absolute_error: 1.0129 - val_loss: 1.0324 - val_mean_absolute_error: 1.4104
Epoch 13/100
193/193 [==============================] - 5s 27ms/step - loss: 0.6662 - mean_absolute_error: 1.0202 - val_loss: 1.0161 - val_mean_absolute_error: 1.3968
Epoch 14/100
193/193 [==============================] - 5s 27ms/step - loss: 0.6571 - mean_absolute_error: 1.0110 - val_loss: 1.0346 - val_mean_absolute_error: 1.4185
Epoch 15/1

In [ ]:
plot([history.history['loss'], history.history['val_loss'], history.history['mean_absolute_error'], history.history['val_mean_absolute_error']], model_title, 
rows=1, cols=2, xaxis_title="epoch", yaxis_title="", subplot_pos=[0, 0, 1, 1], names=["loss", "val_loss", "mean_absolute_error", "val_mean_absolute_error"], subplot_titles=["loss", "mean_absolute_error"])

In [ ]:
load_model_name = "v18"
load_model_title = "CNN-LSTM (6h-to-3h, Huber, Batch 128, 3CNN, Dropout 0.8)"
loaded_model = load_model(f'model/{load_model_name}')

In [ ]:
def inverse_transform_single(data, scaler, axis=0):
  new_scaler = MinMaxScaler()
  new_scaler.min_, new_scaler.scale_ = scaler.min_[axis], scaler.scale_[axis]
  return new_scaler.inverse_transform(data)

In [ ]:
# Retrieve the prediction result (single)
test_sample = test_data
test_shift = 12 * 24
print(f"test_sample shape={test_sample[:n_steps].shape}")
result = loaded_model.predict(np.expand_dims(test_sample[test_shift:n_steps+test_shift], axis=0))
# Inverse transform the normalized data (since the train)
# result = inverse_transform_single(result, mm, 0)
# result = np.append(np.full(n_steps, np.nan), result.flatten())
result = result.flatten()
actual = test_sample[n_steps+test_shift:n_steps+n_horizon+test_shift, 0]
# actual = inverse_transform_single(actual.reshape(-1, 1), mm, 0)
actual = actual.flatten()
print(f"result shape={result.shape}")
print(f"actual shape={actual.shape}")
plot([result, actual], load_model_title, names=['Predictions', 'Actuals'], subplot_pos=[0, 0], yaxis_range=[0, 30])

test_sample shape=(72, 4)
1/1 [==============================] - 0s 22ms/step
result shape=(36,)
actual shape=(36,)


In [ ]:
def make_input_windows(data, n_steps, n_horizon):
  # Generate feature input windows for full prediction over the actual results
  ds = tf.data.Dataset.from_tensor_slices(data)
  # Create the window combined the steps and horizon
  ds = ds.window(n_steps, shift=n_horizon, drop_remainder=True)
  # window() return nested dataset of windows but a regular dataset containing tensors is needed
  ds = ds.flat_map(lambda x : x.batch(n_steps))
  # Raise the dimension of the output
  ds = ds.batch(1)
  
  return ds

In [ ]:
# Full forecasting
test_sample_full = test_data
input_full = make_input_windows(test_sample_full, n_steps, n_horizon)
result_full = loaded_model.predict(input_full)
result_full = result_full.flatten()
actual_full = test_sample_full[n_steps:, 0]
actual_full = actual_full.flatten()
actual_full = np.pad(actual_full, (0, result_full.size - actual_full.size), 'constant', constant_values=(np.nan,))
print(f"result shape={result_full.shape}")
print(f"actual shape={actual_full.shape}")
plot([result_full, actual_full], f"{load_model_title} (Full)", names=['Predictions', 'Actuals'], subplot_pos=[0, 0], yaxis_range=[0, 30])

97/97 [==============================] - 0s 4ms/step
result shape=(3492,)
actual shape=(3492,)


In [ ]:
results = loaded_model.evaluate(train)
print(f"test loss={results[0]}, test acc={results[1]}")
results = loaded_model.evaluate(val)
print(f"val loss={results[0]}, test acc={results[1]}")
results = loaded_model.evaluate(test)
print(f"test loss={results[0]}, test acc={results[1]}")

193/193 [==============================] - 4s 22ms/step - loss: 0.8186 - mean_absolute_error: 1.1866
test loss=0.8186173439025879, test acc=1.1865968704223633
55/55 [==============================] - 1s 19ms/step - loss: 0.9689 - mean_absolute_error: 1.3440
val loss=0.9688965678215027, test acc=1.3440475463867188
27/27 [==============================] - 1s 18ms/step - loss: 1.1331 - mean_absolute_error: 1.5144
test loss=1.1330890655517578, test acc=1.5144097805023193
